In [ ]:
from IPython.display import clear_output
clear_output(wait=True)


In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time
from tqdm import tqdm
import pandas as pd

In [ ]:
browser = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

In [ ]:
browser.get("https://pmsvanidhi.mohua.gov.in/Schemes/SearchVendor")

In [ ]:
state = browser.find_element(By.ID, "StateId")

In [ ]:
#Enter the state
stateName = "GUJARAT"
state.send_keys(stateName)

In [ ]:
#Enter the ULB Names in a list
ULBNAMES = ['Ahmedabad']

In [ ]:
def extract_info(li):
    
    data = {}

    # Extracting SRN No
    srn_no = li.find_element(By.XPATH, ".//div[@class='card-inner']/h5").text
    data["SRN No"] = srn_no

    # Extracting SV Name
    sv_name = li.find_element(By.XPATH, ".//th[text()='SV Name']/../following-sibling::tr[1]/td[1]").text
    data["SV Name"] = sv_name.strip()

    # Extracting Father’s Name/ Spouse Name
    father_or_spouse_name = li.find_element(By.XPATH, ".//th[text()='Father’s Name/ Spouse Name']/../following-sibling::tr[1]/td[2]").text
    data["Father’s Name/ Spouse Name"] = father_or_spouse_name.strip()
    
    collapseButton = li.find_element(By.CLASS_NAME, "accordion-btn")
    collapseButton.click()
    time.sleep(0.5)

    # Extracting ULB Name, Gender, Age, DOB
    second_row = li.find_element(By.CLASS_NAME, "collapse").find_element(By.XPATH, ".//div/table/tbody/tr[2]")
    items = second_row.find_elements(By.TAG_NAME, "td")
    values = [item.text for item in items] 
    
    data["ULB Name"] = values[0]
    data["Gender"] = values[1]
    data["Age"] = values[2]
    data["DOB"] = values[3]

    return data


In [ ]:
allData = []

In [ ]:

for ULBNAME in ULBNAMES:
    ulbName = browser.find_element(By.ID, "_ULBName")
    ulbName.send_keys(ULBNAME)

    searchButton = browser.find_element(By.CLASS_NAME, "ser-btn")
    searchButton.click()
    
    time.sleep(5)


    pageNo = browser.find_element(By.XPATH, './html/body/section[2]/div/div/div[2]/form/div/div[2]/table/tbody/tr/td/div/ul/li[3]').text.strip().split()[-1]
    pageNo = int(pageNo)


    for currPage in tqdm(range(pageNo)):

        listItems = browser.find_elements(By.CLASS_NAME, "search-info-card")
        if not listItems:
            break

        for listItem in listItems:
            try:
                data = extract_info(listItem)
                data['Page'] = str(currPage+1)
                allData.append(data)
            except Exception as e:
                print(f"Error extracting data: {e}")

        try:
            nextButton = browser.find_element(By.XPATH, './html/body/section[2]/div/div/div[2]/form/div/div[2]/table/tbody/tr/td/div/ul/li[4]/a')
            nextButton.click()

            time.sleep(10)

            # Wait for the new page to load
            WebDriverWait(browser, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "search-info-card")))
        except Exception as e:
            print(f"Error navigating to next page: {e}")




In [ ]:
df = pd.DataFrame(allData)

In [ ]:
df.to_csv('vendor_list.csv', index=False)